In [57]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [15]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="사용자가 공개적으로 거래되는 회사에 대해 조사하고 주식을 구매해야 할지 여부를 결정하도록 돕습니다.",
#     model="gpt-3.5-turbo",
#     tools=functions,
# )
assistant_id = "asst_pchemeRDurnq0XNMF6WsM2Sp"

In [50]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "애플 주식을 살까? 말까?",
        }
    ]
)
thread

Thread(id='thread_C9dWr47ERmcARvXGMAVM19lw', created_at=1714133063, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [28]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_ffZKnTPV3GY7IB5QUfsMWFDP', assistant_id='asst_UyCxDLOJFUkDJTgvBVfRlOS2', cancelled_at=None, completed_at=None, created_at=1714199159, expires_at=1714199759, failed_at=None, incomplete_details=None, instructions='You help users with their question on the files they upload.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_DrhQFGPWY0BCj8UD2QiAp80B', tool_choice='auto', tools=[FileSearchTool(type='file_search')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [3]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Book Assistant",
#     instructions="You help users with their question on the files they upload.",
#     model="gpt-3.5-turbo",
#     tools=[{"type": "file_search"}],
# )
assistant_id = "asst_UyCxDLOJFUkDJTgvBVfRlOS2"

In [5]:
thread = client.beta.threads.create(
    messages=[{"role": "user", "content": "I want you to help me with this file"}]
)
thread

Thread(id='thread_DrhQFGPWY0BCj8UD2QiAp80B', created_at=1714198145, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [7]:
file = client.files.create(
    file=client.file_from_path("./examples/chapter_one.pdf"), purpose="assistants"
)
file

FileObject(id='file-PwuF7EbQtpnjU0CEF89OnWwV', bytes=87426, created_at=1714198312, filename='chapter_one.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [14]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="this is my file.",
    attachments=[
        {
            "file_id": file.id,
            "tools": [
                {"type": "file_search"},
            ],
        }
    ],
)

Message(id='msg_gT20Npn7iActal0zz6oDYQ1J', assistant_id=None, attachments=[Attachment(file_id='file-PwuF7EbQtpnjU0CEF89OnWwV', tools=[FileSearchTool(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='this is my file.'), type='text')], created_at=1714198866, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_DrhQFGPWY0BCj8UD2QiAp80B')

In [35]:
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")
        for annotation in message.content[0].text.annotations:
            print(annotation.start_index, annotation.end_index)


def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(run_id=run_id, thread_id=thread_id)


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content,
    )

In [30]:
get_run(run.id, thread.id).status

'completed'

In [36]:
get_messages(thread.id)

user: I want you to help me with this file
user: this is my file.
assistant: I see that the file you uploaded contains excerpts from "1984" by George Orwell. The passages describe the oppressive and manipulative control exerted by the Party, particularly through tools like the Two Minutes Hate and the constant surveillance by the Thought Police. The novel paints a bleak picture of a totalitarian regime where even thoughts are policed and individuality suppressed. If you have any specific questions or need clarification on any particular aspect of the text, feel free to ask!
user: 윈스턴의 집은 무엇인가요? 그리고 그 집은 어떻게 생겼나요?
assistant: 윈스턴의 집은 Victory Mansions이라는 아파트 건물입니다. 그의 아파트는 7층에 위치해 있으며, 아파트 복도에는 거대한 얼굴이 그려진 포스터가 붙어 있습니다. 이 아파트는 파티주의 사회에서 일상적인 생활을 보여주는 곳으로, 왕성한 감시와 제어가 느껴집니다. 집의 분위기는 억지로 조성된 낡고 심한 향기를 풍기는 환경으로 묘사되어 있습니다. 또한, 아파트는 지속적인 감시와 흑흑한 분위기 속에서 살아가고 있는 주인공 윈스턴의 인생을 반영하고 있습니다.【9:0†source】.
257 269


In [25]:
send_message(thread.id, "윈스턴의 집은 무엇인가요? 그리고 그 집은 어떻게 생겼나요?")

Message(id='msg_kQW8dfh3SgLT2imyIc6xfAbp', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='윈스턴의 집은 무엇인가요? 그리고 그 집은 어떻게 생겼나요?'), type='text')], created_at=1714199124, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_DrhQFGPWY0BCj8UD2QiAp80B')